# Cryptocurrency Clusters

In [22]:
#imports
import pandas as pd

## Data Preparation

In [23]:
#read csv
crypto_file="crypto_data.csv"

In [24]:
#Load data
df=pd.read_csv(crypto_file)
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [25]:
#Discard all cryptocurrencies that are not being traded

In [26]:
#Remove all rows that have at least one null value.

In [27]:
#Filter for cryptocurrencies that have been mined. That is, the total coins mined should be greater than zero.

In [28]:
#In order for your dataset to be comprehensible to a machine learning algorithm, its data should be numeric. 
#Since the coin names do not contribute to the analysis of the data, 
#delete the CoinName from the original dataframe.

In [29]:
#Your next step in data preparation is to convert the remaining features with text values, Algorithm and ProofType, 
#into numerical data. To accomplish this task, use Pandas to create dummy variables. Examine the number of rows and 
#columns of your dataset now. How did they change?

In [ ]:
#Standardize your dataset so that columns that contain larger values do not unduly influence the outcome.

## Dimensionality Reduction

In [31]:
#Creating dummy variables above dramatically increased the number of features in your dataset. Perform dimensionality 
#reduction with PCA. Rather than specify the number of principal components when you instantiate the PCA model, it is 
#possible to state the desired explained variance. For example, say that a dataset has 100 features. Using 
#PCA(n_components=0.99) creates a model that will preserve approximately 99% of the explained variance, whether that 
#means reducing the dataset to 80 principal components or 3. For this project, preserve 90% of the explained variance 
#in dimensionality reduction. How did the number of the features change?

In [32]:
#Next, further reduce the dataset dimensions with t-SNE and visually inspect the results. In order to accomplish this 
#task, run t-SNE on the principal components: the output of the PCA transformation. Then create a scatter plot of the 
#t-SNE output. Observe whether there are distinct clusters or not.

## Cluster analysis with k-means

In [ ]:
#Create an elbow plot to identify the best number of clusters. Use a for-loop to determine the inertia for each k 
#between 1 through 10. Determine, if possible, where the elbow of the plot is, and at which value of k it 
#appears.

## Recommendation 

In [35]:
#Based on your findings, make a brief (1-2 sentences) recommendation to your clients. Can the cryptocurrencies be 
#clustered together? If so, into how many clusters?